# 🔥 Module 3: การวิเคราะห์ข้อมูลไฟป่าด้วย Google Earth Engine
## Wildfire Analysis with GEE Python API

**Workshop: การใช้ GEE วิเคราะห์ข้อมูลอุตุ-อุทก-ไฟป่า-คุณภาพน้ำ**

---

### 📋 เนื้อหาในบทนี้
1. MODIS Active Fire (จุดความร้อน)
2. VIIRS Active Fire (ความละเอียดสูงกว่า)
3. MODIS Burned Area (พื้นที่เผาไหม้)
4. NBR & dNBR - ดัชนีความรุนแรงไฟ
5. Time Series ไฟป่าภาคเหนือ
6. ความสัมพันธ์ไฟป่ากับ PM2.5

### 📌 พื้นที่ศึกษา: ภาคเหนือของไทย (เชียงใหม่, เชียงราย, แม่ฮ่องสอน)
### ⏱️ เวลา: ~35 นาที (10:45 - 11:20)

---
## 1. Setup & กำหนดพื้นที่ศึกษา

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ee.Authenticate()
ee.Initialize(project='YOUR-PROJECT-ID')  # <-- ใส่ GEE Cloud Project ID

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 100
print('GEE เชื่อมต่อสำเร็จ ✅')

In [ ]:
# พื้นที่ศึกษา: 9 จังหวัดภาคเหนือตอนบน
provinces = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

north_provinces = ['Chiang Mai', 'Chiang Rai', 'Mae Hong Son',
                   'Lampang', 'Lamphun', 'Phayao',
                   'Phrae', 'Nan', 'Uttaradit']

northern_region = provinces.filter(
    ee.Filter.inList('ADM1_NAME', north_provinces)
)

chiang_mai = provinces.filter(ee.Filter.eq('ADM1_NAME', 'Chiang Mai'))

Map = geemap.Map(center=[18.5, 99.5], zoom=8)
Map.addLayer(northern_region, {'color': 'green'}, 'Northern Region')
Map.addLayer(chiang_mai, {'color': 'red'}, 'Chiang Mai')
Map

---
## 2. MODIS Active Fire (Thermal Anomalies)

**MODIS/Terra Thermal Anomalies & Fire**  
- ตรวจจุดความร้อน (Hotspot) จากดาวเทียม Terra & Aqua  
- ความละเอียด: 1 km  
- Band: `FireMask` (7=low, 8=nominal, 9=high confidence)  
- Dataset: `MODIS/061/MOD14A1` (Terra), `MODIS/061/MYD14A1` (Aqua)

In [ ]:
# โหลด MODIS Active Fire - ช่วงหน้าไฟ (ม.ค. - เม.ย. 2024)
modis_fire = ee.ImageCollection('MODIS/061/MOD14A1') \
    .filterDate('2024-01-01', '2024-04-30') \
    .filterBounds(northern_region) \
    .select('FireMask')

print(f'จำนวนภาพ MODIS Fire: {modis_fire.size().getInfo()} ภาพ')

# สร้างแผนที่รวมจุดไฟทั้งหมด (max composite)
# FireMask >= 7 = fire detected
fire_composite = modis_fire.max().clip(northern_region)
fire_mask = fire_composite.gte(7).selfMask()

Map2 = geemap.Map(center=[18.5, 99.5], zoom=8)
Map2.addLayer(fire_mask, {'palette': ['orange', 'red']}, 'MODIS Fire Hotspots (Jan-Apr 2024)')
Map2.addLayer(northern_region, {'color': 'white'}, 'Boundary', opacity=0.5)
Map2

---
## 3. VIIRS Active Fire (ละเอียดกว่า)

**VIIRS** (Visible Infrared Imaging Radiometer Suite)  
- ความละเอียด: 375 m (ละเอียดกว่า MODIS 1 km)  
- ดาวเทียม: Suomi NPP  
- Dataset: `NASA/FIRMS` (Fire Information for Resource Management System)

In [ ]:
# FIRMS (Fire Information for Resource Management System)
firms = ee.ImageCollection('FIRMS') \
    .filterDate('2024-01-01', '2024-04-30') \
    .filterBounds(northern_region) \
    .select('T21')  # Brightness Temperature at 21 μm

# จุดที่มีอุณหภูมิสูง = ไฟ
fire_temp = firms.max().clip(northern_region)
fire_high = fire_temp.gt(330).selfMask()  # > 330K = fire

Map3 = geemap.Map(center=[18.5, 99.5], zoom=8)
Map3.addLayer(fire_temp, {'min': 300, 'max': 400, 'palette': ['yellow', 'orange', 'red', 'darkred']},
             'FIRMS Temperature (K)')
Map3.addLayer(fire_high, {'palette': ['red']}, 'High Temperature Fire')
Map3

---
## 4. MODIS Burned Area

**MODIS Burned Area (MCD64A1)**  
- ให้ข้อมูลพื้นที่เผาไหม้รายเดือน  
- ความละเอียด: 500 m  
- Band: `BurnDate` (วันที่เผาไหม้, Julian day)  
- Dataset: `MODIS/061/MCD64A1`

In [ ]:
# โหลด MODIS Burned Area ม.ค.-เม.ย. 2024
burned = ee.ImageCollection('MODIS/061/MCD64A1') \
    .filterDate('2024-01-01', '2024-04-30') \
    .filterBounds(northern_region) \
    .select('BurnDate')

# สร้าง composite พื้นที่เผาไหม้ทั้งหมด
burned_composite = burned.max().clip(northern_region)
burned_mask = burned_composite.gt(0).selfMask()

vis_burned = {
    'min': 1,
    'max': 120,  # Julian day (1 ม.ค. - 30 เม.ย.)
    'palette': ['yellow', 'orange', 'red', 'darkred', 'black']
}

Map4 = geemap.Map(center=[18.5, 99.5], zoom=8)
Map4.addLayer(burned_composite.selfMask(), vis_burned, 'Burn Date (Julian Day)')
Map4.addLayer(burned_mask, {'palette': ['red']}, 'Total Burned Area', shown=False)
Map4.add_colorbar(vis_burned, label='Burn Date (Julian Day)',
                  layer_name='Burn Date (Julian Day)')
Map4

In [ ]:
# คำนวณพื้นที่เผาไหม้แต่ละจังหวัด
results = []

for prov_name in north_provinces:
    prov = provinces.filter(ee.Filter.eq('ADM1_NAME', prov_name))
    burned_prov = burned_mask.unmask(0).clip(prov)

    area = burned_prov.multiply(ee.Image.pixelArea()).divide(1e6)  # ตร.กม.
    total = area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=prov.geometry(),
        scale=500,
        maxPixels=1e10
    ).getInfo()

    results.append({'Province': prov_name, 'Burned_Area_km2': total.get('BurnDate', 0)})

df_burned = pd.DataFrame(results).sort_values('Burned_Area_km2', ascending=False)
print(df_burned.to_string(index=False))

In [ ]:
# กราฟพื้นที่เผาไหม้ตามจังหวัด
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(df_burned['Province'], df_burned['Burned_Area_km2'],
               color='orangered', edgecolor='darkred')
ax.set_xlabel('Burned Area (km²)', fontsize=12)
ax.set_title('Burned Area by Province - Northern Thailand (Jan-Apr 2024)',
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

for bar in bars:
    width = bar.get_width()
    ax.text(width + 5, bar.get_y() + bar.get_height()/2.,
            f'{width:.0f}', ha='left', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 5. NBR & dNBR - ดัชนีความรุนแรงของไฟ

**NBR** (Normalized Burn Ratio)
$$NBR = \frac{NIR - SWIR2}{NIR + SWIR2}$$

**dNBR** (differenced NBR) = NBR_before - NBR_after

| dNBR | Burn Severity |
|------|---------------|
| < 0.1 | Unburned |
| 0.1 - 0.27 | Low |
| 0.27 - 0.44 | Moderate-Low |
| 0.44 - 0.66 | Moderate-High |
| > 0.66 | High |

In [ ]:
# คำนวณ dNBR สำหรับเชียงใหม่
def get_s2_composite(start, end, aoi):
    """สร้าง Sentinel-2 median composite"""
    return ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
        .median()

def calc_nbr(image):
    """คำนวณ NBR จาก Sentinel-2"""
    return image.normalizedDifference(['B8', 'B12']).rename('NBR')

# ก่อนไฟ (พ.ย. - ธ.ค. 2023)
pre_fire = get_s2_composite('2023-11-01', '2023-12-31', chiang_mai)
nbr_pre = calc_nbr(pre_fire)

# หลังไฟ (เม.ย. - พ.ค. 2024)
post_fire = get_s2_composite('2024-04-01', '2024-05-31', chiang_mai)
nbr_post = calc_nbr(post_fire)

# dNBR
dnbr = nbr_pre.subtract(nbr_post).rename('dNBR').clip(chiang_mai)

vis_dnbr = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['green', 'yellow', 'orange', 'red', 'darkred']
}

Map5 = geemap.Map(center=[18.8, 98.98], zoom=10)
Map5.addLayer(dnbr, vis_dnbr, 'dNBR (Burn Severity)')
Map5.add_colorbar(vis_dnbr, label='dNBR', layer_name='dNBR (Burn Severity)')
Map5

In [ ]:
# จำแนกความรุนแรงของไฟ
severity = ee.Image(0) \
    .where(dnbr.gte(0.1).And(dnbr.lt(0.27)), 1) \
    .where(dnbr.gte(0.27).And(dnbr.lt(0.44)), 2) \
    .where(dnbr.gte(0.44).And(dnbr.lt(0.66)), 3) \
    .where(dnbr.gte(0.66), 4) \
    .clip(chiang_mai) \
    .selfMask()

severity_vis = {
    'min': 1,
    'max': 4,
    'palette': ['yellow', 'orange', 'red', 'darkred']
}

Map6 = geemap.Map(center=[18.8, 98.98], zoom=10)
Map6.addLayer(severity, severity_vis, 'Burn Severity Class')
Map6

---
## 6. Time Series: จำนวนจุดความร้อนรายเดือน (หลายปี)

In [ ]:
# นับจุดความร้อนรายเดือน 2020-2024
years = range(2020, 2025)
months = range(1, 13)

fire_counts = []

for year in years:
    for month in months:
        start = f'{year}-{month:02d}-01'
        if month == 12:
            end = f'{year+1}-01-01'
        else:
            end = f'{year}-{month+1:02d}-01'

        monthly_fire = ee.ImageCollection('MODIS/061/MOD14A1') \
            .filterDate(start, end) \
            .filterBounds(northern_region) \
            .select('FireMask')

        # นับ pixel ที่มีไฟ
        fire_img = monthly_fire.max().gte(7).unmask(0)
        count = fire_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=northern_region.geometry(),
            scale=1000,
            maxPixels=1e10
        ).getInfo()

        fire_counts.append({
            'year': year,
            'month': month,
            'fire_pixels': count.get('FireMask', 0)
        })

df_fire = pd.DataFrame(fire_counts)
print('ดึงข้อมูลสำเร็จ ✅')
df_fire.head()

In [ ]:
# กราฟเปรียบเทียบจุดความร้อนรายเดือน หลายปี
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, ax = plt.subplots(figsize=(14, 6))

colors = ['#2196F3', '#4CAF50', '#FF9800', '#F44336', '#9C27B0']
for i, year in enumerate(years):
    year_data = df_fire[df_fire['year'] == year]
    ax.plot(month_labels, year_data['fire_pixels'].values, 'o-',
            linewidth=2, markersize=6, color=colors[i], label=str(year))

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Fire Pixel Count', fontsize=12)
ax.set_title('Monthly Fire Hotspots - Northern Thailand (2020-2024)',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

# เน้นช่วงหน้าไฟ
ax.axvspan(0.5, 3.5, alpha=0.1, color='red', label='Fire Season')
ax.text(2, ax.get_ylim()[1]*0.95, 'Fire Season', ha='center',
        fontsize=11, color='red', fontstyle='italic')

plt.tight_layout()
plt.show()

---
## 7. ไฟป่ากับ PM2.5 (Sentinel-5P)

ดูความสัมพันธ์ระหว่างจุดความร้อนกับ Aerosol Index (ตัวชี้วัดมลพิษ)

In [ ]:
# Sentinel-5P Aerosol Index (ช่วงหน้าไฟ มี.ค. 2024)
s5p_ai = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
    .filterDate('2024-03-01', '2024-03-31') \
    .filterBounds(northern_region) \
    .select('absorbing_aerosol_index') \
    .mean() \
    .clip(northern_region)

vis_ai = {
    'min': -1,
    'max': 3,
    'palette': ['green', 'yellow', 'orange', 'red', 'purple']
}

Map7 = geemap.Map(center=[18.5, 99.5], zoom=8)
Map7.addLayer(s5p_ai, vis_ai, 'Aerosol Index (Mar 2024)')
Map7.addLayer(fire_mask, {'palette': ['red']}, 'Fire Hotspots')
Map7.add_colorbar(vis_ai, label='Aerosol Index',
                  layer_name='Aerosol Index (Mar 2024)')
Map7

---
## 📝 สรุป Module 3

| หัวข้อ | Dataset | Band / วิธี | ความละเอียด |
|--------|---------|-------------|------------|
| จุดความร้อน | MODIS MOD14A1 | FireMask ≥ 7 | 1 km |
| จุดความร้อน | FIRMS | T21 > 330K | 1 km |
| พื้นที่เผาไหม้ | MODIS MCD64A1 | BurnDate | 500 m |
| ความรุนแรงไฟ | Sentinel-2 | dNBR | 10-20 m |
| มลพิษอากาศ | Sentinel-5P | Aerosol Index | ~5.5 km |

### 💡 Tips
- ภาคเหนือไทยมีฤดูไฟป่าหลักคือ ม.ค. - เม.ย.
- MODIS ดีสำหรับ monitoring ระดับภูมิภาค
- Sentinel-2 + dNBR เหมาะสำหรับวิเคราะห์ความรุนแรงในพื้นที่เฉพาะ
- ไฟป่ามีความสัมพันธ์โดยตรงกับ PM2.5 ในภาคเหนือ

---
**➡️ ต่อไป: Module 4 - คุณภาพน้ำ (Water Quality)**